ANALISIS RAPIDO

In [11]:
import sys
from pathlib import Path
import pandas as pd

# Agregamos la carpeta src al path para poder importar
path_inicio = Path.cwd().parent/"src"
sys.path.append(str(path_inicio))
from utilidades.constantes import DATA_PATH, DATA_PROCESADA_PATH, PATRON_MOSTRADOR_PATH, MOSTRADOR_SALIDA_PATH, PATRON_FACTURACION_PATH, FACTURACION_SALIDA_PATH, ARCHIVO_UNIFICADO_PATH


In [12]:
df_unificado = pd.read_csv(ARCHIVO_UNIFICADO_PATH, encoding='utf-8-sig')
df_mostrador = pd.read_csv(MOSTRADOR_SALIDA_PATH, encoding='utf-8-sig')
df_facturacion = pd.read_csv(FACTURACION_SALIDA_PATH, encoding='utf-8-sig')

In [13]:
# Sumar ventas en df_mostrador solo para 2023
ventas_mostrador_2023 = df_mostrador[df_mostrador['Fecha'].str.startswith('2023')]
suma_mostrador_2023 = ventas_mostrador_2023['TotalVenta'].sum()
print(f"Suma total mostrador 2023: {suma_mostrador_2023}")

# Sumar ventas en df_facturacion solo para 2023
ventas_facturacion_2023 = df_facturacion[df_facturacion['fecha_emision'].str.startswith('2023')]
suma_facturacion_2023 = ventas_facturacion_2023['importe_total'].sum()
print(f"Suma total facturación 2023: {suma_facturacion_2023}")

# Suma total entre ambos
suma_total_2023 = suma_mostrador_2023 + suma_facturacion_2023
print(f"Suma total mostrador + facturación 2023: {suma_total_2023}")


Suma total mostrador 2023: 191148270
Suma total facturación 2023: 628939484
Suma total mostrador + facturación 2023: 820087754
